In [3]:
from transformers import AutoTokenizer

In [4]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [5]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


In [8]:
print(inputs['input_ids'].shape)
print(inputs['attention_mask'].shape)


torch.Size([2, 16])
torch.Size([2, 16])


In [16]:
input_shape = inputs['input_ids'].size()
input_shape


torch.Size([2, 16])

In [18]:
import torch
# extended_attention_mask: torch.Tensor = get_extended_attention_mask(
#             attention_mask, input_shape, device)
def create_extended_attention_mask_for_decoder(input_shape, attention_mask, device):
        batch_size, seq_length = input_shape
        seq_ids = torch.arange(seq_length, device=device)
        causal_mask = seq_ids[None, None, :].repeat(batch_size, seq_length, 1) <= seq_ids[None, :, None]
        # in case past_key_values are used we need to add a prefix ones mask to the causal mask
        # causal and attention masks must have same type with pytorch version < 1.3
        causal_mask = causal_mask.to(attention_mask.dtype)

        if causal_mask.shape[1] < attention_mask.shape[1]:
            prefix_seq_len = attention_mask.shape[1] - causal_mask.shape[1]
            causal_mask = torch.cat(
                [
                    torch.ones((batch_size, seq_length, prefix_seq_len), device=device, dtype=causal_mask.dtype),
                    causal_mask,
                ],
                axis=-1,
            )

        extended_attention_mask = causal_mask[:, None, :, :] * attention_mask[:, None, None, :]
        return extended_attention_mask

In [21]:
attention_mask = inputs['attention_mask'] 
extended_attention_mask = create_extended_attention_mask_for_decoder(
                    input_shape, attention_mask,device='cpu')

In [24]:
print(extended_attention_mask.shape)
extended_attention_mask

torch.Size([2, 1, 16, 16])


tensor([[[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
          [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
          [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
          [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
          [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
          [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
          [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
          [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
          [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]],


        [[[1, 0, 0, 0, 0, 0, 0, 0, 0

In [25]:
input_ids = inputs['input_ids']
input_shape = input_ids.size()
input_shape

torch.Size([2, 16])

In [28]:
B, token_lenth= input_ids.size()
input_shape = torch.Size([B,token_lenth])
input_shape

torch.Size([2, 16])

In [39]:
B, C, HW, HW = 24,3,3224,224
num_tokens = 196 + 2 # for 14 * 14
attention_mask = torch.ones(B,num_tokens).to(torch.float32)#.double()
attention_mask.dtype

torch.float32

In [43]:
attention_mask =attention_mask.to(torch.float32)
attention_mask.dtype

torch.float32

In [44]:
attention_mask =attention_mask.to(torch.float64)
attention_mask.dtype

torch.float64

In [2]:
import torch
import args
from args import AdditionalArguments, DataTrainingArguments
args_addi= torch.load('/home/chengquan/ToP-prune_before_FFN/train_out_put_dir/test_run_2/additional_args.bin',map_location="cpu")

NameError: name 'basestring' is not defined